# Linear Programming

Linear programming is a mathematical technique to achieve the best outcome (such as maximum profit, or least possible cost), whilst having various constraints.

For our course, we will be using the `pulp` library which you can find more information __[here](https://www.coin-or.org/PuLP/pulp.html)__.

## Problem 1
*Suppose a farmer has 75 acres on which to plant two crops: wheat and barley. To produce these crops, it costs the farmer (for seed, fertilizer, etc.) $120 per acre for the wheat and $210 per acre for the barley. The farmer has $15000 available for expenses. But after the harvest, the farmer must store the crops while awaiting favorable market conditions. The farmer has storage space for 4000 bushels.
Each acre yields an average of 110 bushels of wheat or 30 bushels of barley. If the net profit per bushel of wheat (after all expenses have been subtracted) is $1.30 and for barley is $2.00, how should the farmer plant the 75 acres to maximize profit?*

## Setting up the Problem

The first task is to set up the problem by:
- Creating an `LpProblem`
- Defining the variables (including any bounds, and data type)

## Variable declerations

* w represents acres of wheat
* b represents acres of barley
* p represents total profit

### Objective function

* max(p): p = 1.3 * 110 * w + 2 * 20 * b

### Constraints

* 120w + 210b <= 15000

* 110w + 30b <= 4000


### Bounds

* w >= 0
* w <= 75
* b >= 0
* b <= 75


In [37]:
# import libraries
from pulp import *

# create the linear programming model
lp = LpProblem("Wheat", LpMaximize)

# define variables
w = LpVariable(name='Wheat', lowBound=0, upBound=75, cat='Integer')
b = LpVariable(name='Barley', lowBound=0, upBound=75, cat='Integer')

## Determine the Objective

Next step is to determine the objective

In this case we have: 
- barley that has a net profit of 2 and every acre yields 30 bushels; hence we have $b*2*30$
- wheat that has a net profit of 1.3 and every acre yields 110 bushels; hence we have $w*1.3*110$

We want to maximize the profit, so we have $max(p) = b*2*30 + w*1.3*110$

This can be coded as following

In [38]:
# specify objective function
lp.objective = (1.3*110*w) + (2*30*b)
lp

Wheat:
MAXIMIZE
60*Barley + 143.0*Wheat + 0.0
VARIABLES
0 <= Barley <= 75 Integer
0 <= Wheat <= 75 Integer

## Setting up the Constraints

The next step is to plot out the constraints. In this case we have:
- budget - The total budget is $15,000, wheat costs $120 per acre, and barley costs $210 per acre. The total cannot exceed the available budget hence: $120*w + 210*b <= 15000$
- storage - The total storage available is 4000, wheat needs 110 per acre, and barley 30 per acre. The total bushels cannot exceed the available storage, hence: $110*w + 30*b <=4000$
- fields - The total acreage of fields available is 75, hence the total of wheat and barley cannot exceed it: $w+b<=75$

In [39]:
# specify constraints
lp.addConstraint(w+b<=75, 'area')
lp.addConstraint((120*w)+(210*b)<=15000, 'budget')
lp.addConstraint((110*w)+(30*b)<=4000, 'space')
lp.constraints 

OrderedDict([('area', 1*Barley + 1*Wheat + -75 <= 0),
             ('budget', 210*Barley + 120*Wheat + -15000 <= 0),
             ('space', 30*Barley + 110*Wheat + -4000 <= 0)])

## Solving the Problem

The last step is to solve the problem. This can be done by calling the `solve` function.

In [40]:
# solve the linear programming problem
# 1 = Optimal, 2 = Infeasible, 3 = Unbounded, 4 = Undefined, 5 = Not Solved

status = lp.solve(PULP_CBC_CMD(msg=0))
print(status)

# print solution
for var in lp.variables():
    print(var,'=',value(var))
    
print('OPT',value(lp.objective))

1
Barley = 52.0
Wheat = 22.0
OPT 6266.0


If we look at the outcome we have:
- status = 1 - The staus is optimal (it can be not solved, infeasible, unbounded or undef)
- Barley = 53.125 - The farmer should sow barley in 53.125 acres of fields
- Wheat = 21.875 - The farmer should sow wheat in 21.875 acres of fields
- Opt = 6315.625 - If the farmer sows the recommened distribution he would earn $6315.63